In [148]:
from bs4 import BeautifulSoup
import requests
import json
import spotipy 

import random 
from time import sleep

import pandas as pd

## Scraping the Billbord Website for 100 Hot songs

In [161]:
#------scrapping 2021 data-------#

url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
newest_soup = BeautifulSoup(response.content, 'html.parser')

In [162]:
response.status_code

200

In [164]:
#--------2021 song names--------#

newest_song = newest_soup.select('span.chart-element__information__song.text--truncate.color--primary')
newest_song = [t.text for t in newest_song]


#--------2021 artist names--------#

newest_artist = newest_soup.select('span.chart-element__information__artist.text--truncate.color--secondary')
newest_artist = [t.text for t in newest_artist]

In [166]:
#------scrapping 2019,2020 data-------#

str = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs'
years = [ 2006, 2007 ,2008, 2009, 2010,2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018 ,2019, 2020]

song_pool_soups = []
artist_pool_soups = []

for year in years:
    url = str.format(year)
    response = requests.get(url)
    #print(response.status_code)
    
    soup = BeautifulSoup(response.content)
    song_pool_soups.append(soup.find_all('div', class_ = 'ye-chart-item__title'))
    artist_pool_soups.append(soup.find_all('div', class_ = 'ye-chart-item__artist'))
    
    
    wait_time = random.randint(1,4)
    sleep(wait_time)

In [167]:
#------songs & artists 2006-2020------#

for i in range(0,len(years)):
    song_pool_soups[i] = [t.text.replace('\n','') for t in song_pool_soups[i]]
    artist_pool_soups[i] = [t.text.replace('\n','') for t in artist_pool_soups[i]]    

In [ ]:
#release_year = soup.find('span', class_ ='dropdown__year-select-date').get_text()
#song_release_year = [2006]*100 + [2007]*100 + [2008]*100 + [2009]*100 + [2010]*100 + [2011]*100 + [2012]*100 + [2013]*100 + [2014]*100 + [2015]*100 + [2016]*100 + [2017]*100 + [2018]*100 + [2019]*100 + [2020]*100 + [2021]*100

In [168]:
#----------concatinate the 2006-2020 years data together----------#

song_name = []
song_artist = []

for i in range(0,len(older_song_soups)):
        song_name += older_song_soups[i]
        song_artist += older_artist_soups[i]

In [243]:
#-----current hot topic dataframe 2021----------#

df_current = pd.DataFrame({'song_name': newest_song, 'artist': newest_artist})
df_current.head(5)

,song_name,artist
0,Way 2 Sexy,Drake Featuring Future & Young Thug
1,Girls Want Girls,Drake Featuring Lil Baby
2,Fair Trade,Drake Featuring Travis Scott
3,Champagne Poetry,Drake
4,Knife Talk,Drake Featuring 21 Savage & Project Pat


In [173]:
#----- hot topic dataframe 2006-2020----------#

df = pd.DataFrame({'song_name': song_name, 'artist': song_artist})
df.tail(5)

,song_name,artist
1493,More Than My Hometown,Morgan Wallen
1494,Lovin' On You,Luke Combs
1495,Said Sum,Moneybagg Yo
1496,Slide,H.E.R. Featuring YG
1497,Walk Em Down,NLE Choppa Featuring Roddy Ricch


In [244]:
#---------Making the dataframe content lowercase-----------#

df_current["song_name"] = df_current["song_name"].apply(lambda x: x.lower())
df_current["artist"] = df_current["artist"].apply(lambda x: x.lower())

In [245]:
#--------maching sequence of input with hot songs------------#

from difflib import SequenceMatcher 

similarity = []
def similar(a):
    for i in range (0,len(newest_song)):
        similarity.append(SequenceMatcher(None, a, newest_song[i]).ratio())
        most_similar = similarity.index(max(similarity))
    return newest_song[most_similar]           

'Fair Trade'

In [220]:
#--------Taking input ------------#


def user_song():
    input_song = input('Enter Your Favorite Song: ')
    
    similarity = []
    if input_song.lower() in df_current.song_name.values:
        print('The recommended song for you is: ',random.choice(df_current['song_name']),'\n')
            
    elif similar(input_song.lower()) in df_current.song_name.values:
        print('Do you mean: ', similar(input_song.lower()))
        user_song()
    
    else:
        print('Sorry, Pick another song\n')
        user_song()

In [221]:
user_song()

Enter Your Favorite Song: far trade
Sorry, Pick another song

Enter Your Favorite Song: fair trade
The recommended song for you is:  wockesha 



In [222]:
import config
import pprint

Client_ID = config.client_id
Client_Secret = config.client_secret

In [223]:
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = Client_ID, client_secret = Client_Secret))

In [228]:
song = sp.search(q="comptine d'un autre été", limit=1) 


{'href': 'https://api.spotify.com/v1/search?query=comptine+d%27un+autre+%C3%A9t%C3%A9&type=track&offset=0&limit=1',
 'items': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/00sazWvoTLOqg5MFwC68Um'},
      'href': 'https://api.spotify.com/v1/artists/00sazWvoTLOqg5MFwC68Um',
      'id': '00sazWvoTLOqg5MFwC68Um',
      'name': 'Yann Tiersen',
      'type': 'artist',
      'uri': 'spotify:artist:00sazWvoTLOqg5MFwC68Um'}],
    'available_markets': ['US'],
    'external_urls': {'spotify': 'https://open.spotify.com/album/7JG0QwvJRzPILHEW1xHEJj'},
    'href': 'https://api.spotify.com/v1/albums/7JG0QwvJRzPILHEW1xHEJj',
    'id': '7JG0QwvJRzPILHEW1xHEJj',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b27301e41b3cb7d14f569756801b',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e0201e41b3cb7d14f569756801b',
      'width': 300},
     {'height': 64,
   

In [229]:
song['tracks']['items'][0]

[{'album': {'album_type': 'album',
   'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/00sazWvoTLOqg5MFwC68Um'},
     'href': 'https://api.spotify.com/v1/artists/00sazWvoTLOqg5MFwC68Um',
     'id': '00sazWvoTLOqg5MFwC68Um',
     'name': 'Yann Tiersen',
     'type': 'artist',
     'uri': 'spotify:artist:00sazWvoTLOqg5MFwC68Um'}],
   'available_markets': ['US'],
   'external_urls': {'spotify': 'https://open.spotify.com/album/7JG0QwvJRzPILHEW1xHEJj'},
   'href': 'https://api.spotify.com/v1/albums/7JG0QwvJRzPILHEW1xHEJj',
   'id': '7JG0QwvJRzPILHEW1xHEJj',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab67616d0000b27301e41b3cb7d14f569756801b',
     'width': 640},
    {'height': 300,
     'url': 'https://i.scdn.co/image/ab67616d00001e0201e41b3cb7d14f569756801b',
     'width': 300},
    {'height': 64,
     'url': 'https://i.scdn.co/image/ab67616d0000485101e41b3cb7d14f569756801b',
     'width': 64}],
   'name': 'Amélie (Original Soundtrack)',
   

In [230]:
song = sp.search(q="comptine d'un autre été", limit=1) 

#pprint.pprint(song['tracks']['items'][0]['uri'])

song["tracks"]["items"][0]["uri"]
sp.audio_features(song["tracks"]["items"][0]["uri"])[0]
my_dict = sp.audio_features(song["tracks"]["items"][0]["uri"])[0] # you can provide a list of uri's

print(my_dict)
my_dict_new = { key: [my_dict[key]] for key in list(my_dict.keys()) }
#my_dict_new['name'] = [song["tracks"]["items"][0]['name']]
#print(my_dict_new)

pd.DataFrame(my_dict_new)

{'danceability': 0.484, 'energy': 0.192, 'key': 2, 'loudness': -15.378, 'mode': 1, 'speechiness': 0.0465, 'acousticness': 0.991, 'instrumentalness': 0.908, 'liveness': 0.106, 'valence': 0.0559, 'tempo': 100.084, 'type': 'audio_features', 'id': '4O2N861eOnF9q8EtpH8IJu', 'uri': 'spotify:track:4O2N861eOnF9q8EtpH8IJu', 'track_href': 'https://api.spotify.com/v1/tracks/4O2N861eOnF9q8EtpH8IJu', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4O2N861eOnF9q8EtpH8IJu', 'duration_ms': 140733, 'time_signature': 4}
{'danceability': [0.484], 'energy': [0.192], 'key': [2], 'loudness': [-15.378], 'mode': [1], 'speechiness': [0.0465], 'acousticness': [0.991], 'instrumentalness': [0.908], 'liveness': [0.106], 'valence': [0.0559], 'tempo': [100.084], 'type': ['audio_features'], 'id': ['4O2N861eOnF9q8EtpH8IJu'], 'uri': ['spotify:track:4O2N861eOnF9q8EtpH8IJu'], 'track_href': ['https://api.spotify.com/v1/tracks/4O2N861eOnF9q8EtpH8IJu'], 'analysis_url': ['https://api.spotify.com/v1/audio-analysis/

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.484,0.192,2,-15.378,1,0.0465,0.991,0.908,0.106,0.0559,100.084,audio_features,4O2N861eOnF9q8EtpH8IJu,spotify:track:4O2N861eOnF9q8EtpH8IJu,https://api.spotify.com/v1/tracks/4O2N861eOnF9...,https://api.spotify.com/v1/audio-analysis/4O2N...,140733,4


In [231]:
song = sp.search(q="comptine d'un autre été", limit=1) 

In [235]:
song['tracks']['items'][0]['uri']

'spotify:track:4O2N861eOnF9q8EtpH8IJu'

In [240]:
sp.audio_features(song['tracks']['items'][0]['uri'])[0]

{'danceability': 0.484,
 'energy': 0.192,
 'key': 2,
 'loudness': -15.378,
 'mode': 1,
 'speechiness': 0.0465,
 'acousticness': 0.991,
 'instrumentalness': 0.908,
 'liveness': 0.106,
 'valence': 0.0559,
 'tempo': 100.084,
 'type': 'audio_features',
 'id': '4O2N861eOnF9q8EtpH8IJu',
 'uri': 'spotify:track:4O2N861eOnF9q8EtpH8IJu',
 'track_href': 'https://api.spotify.com/v1/tracks/4O2N861eOnF9q8EtpH8IJu',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4O2N861eOnF9q8EtpH8IJu',
 'duration_ms': 140733,
 'time_signature': 4}